In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_cherry.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation:   0%|                             | 0/30 [00:00<?, ?it/s]/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))
Eval image generation: 100%|████████████████████| 30/30 [00:13<00:00,  2.23it/s]


eval images shape: (30, 1041, 1302) contain 28064 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.00625,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 273.0,
 'bg_width': 2,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4791/Run/pool_CB4791_pos76/phase_cherry_mask_dilated/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4791/Run/pool_CB4791_pos76/edt_noise_cherry.pkl',
 'min_edt': 0,
 'max_edt': 7,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=64, max_iters=15000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 4158.927 || Time Upd.: 812.3 ms  || BatchNr.: 100
 || Cost: 487.560 || Time Upd.: 815.3 ms  || BatchNr.: 200
 || Cost: 218.953 || Time Upd.: 815.8 ms  || BatchNr.: 300
 || Cost: 205.664 || Time Upd.: 810.1 ms  || BatchNr.: 400
 || Cost: 246.484 || Time Upd.: 816.0 ms  || BatchNr.: 500
 || Cost: 149.001 || Time Upd.: 813.2 ms  || BatchNr.: 600
 || Cost: 144.516 || Time Upd.: 815.8 ms  || BatchNr.: 700
 || Cost: 140.774 || Time Upd.: 811.6 ms  || BatchNr.: 800
 || Cost: 121.606 || Time Upd.: 805.7 ms  || BatchNr.: 900
 || Cost: 141.537 || Time Upd.: 814.1 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 28064FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 27723 ,preds: 10741
JoR: 0.683 || Eff_3d: 2.216 || Jaccard: 0.376 || Factor: 755.759 || RMSE_lat: 55.081 || RMSE_ax: 186.903 || RMSE_x

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19861
JoR: 2.515 || Eff_3d: 64.607 || Jaccard: 0.923 || Factor: 717.911 || RMSE_lat: 36.692 || RMSE_ax: 64.766 || RMSE_x: 26.256 || RMSE_y: 25.630 || Cost: 21.539 || Recall: 0.924 || Precision: 0.999 || BatchNr.: 9400 || Time Upd.: 872.8 ms 
 || Count Cost: 27.7824 || Localization Cost: 6.9907 || Bg Cost: 0.0305 || P_locs Cost: 26.2081saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 64.607 || 36.692 || 64.766 || 9400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19724
JoR: 2.516 || Eff_3d: 64.11

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19851
JoR: 2.510 || Eff_3d: 64.477 || Jaccard: 0.923 || Factor: 708.458 || RMSE_lat: 36.751 || RMSE_ax: 65.163 || RMSE_x: 26.065 || RMSE_y: 25.909 || Cost: 20.213 || Recall: 0.924 || Precision: 0.999 || BatchNr.: 10600 || Time Upd.: 880.1 ms 
 || Count Cost: 26.8083 || Localization Cost: 6.0790 || Bg Cost: 0.0294 || P_locs Cost: 25.6851saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 64.477 || 36.751 || 65.163 || 10600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19970
JoR: 2.560 || Eff_3d: 65.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19968
JoR: 2.598 || Eff_3d: 65.717 || Jaccard: 0.927 || Factor: 711.560 || RMSE_lat: 35.691 || RMSE_ax: 62.607 || RMSE_x: 25.389 || RMSE_y: 25.084 || Cost: 20.899 || Recall: 0.929 || Precision: 0.998 || BatchNr.: 11800 || Time Upd.: 876.7 ms 
 || Count Cost: 27.1370 || Localization Cost: 6.6112 || Bg Cost: 0.0296 || P_locs Cost: 25.9831saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.717 || 35.691 || 62.607 || 11800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20076
JoR: 2.546 || Eff_3d: 64.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19906
JoR: 2.523 || Eff_3d: 64.427 || Jaccard: 0.925 || Factor: 702.732 || RMSE_lat: 36.662 || RMSE_ax: 65.744 || RMSE_x: 25.875 || RMSE_y: 25.972 || Cost: 19.938 || Recall: 0.926 || Precision: 0.999 || BatchNr.: 13000 || Time Upd.: 873.1 ms 
 || Count Cost: 27.0683 || Localization Cost: 5.8020 || Bg Cost: 0.0276 || P_locs Cost: 26.0610saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 64.427 || 36.662 || 65.744 || 13000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20172
JoR: 2.558 || Eff_3d: 65.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19784
JoR: 2.566 || Eff_3d: 65.244 || Jaccard: 0.919 || Factor: 673.354 || RMSE_lat: 35.835 || RMSE_ax: 63.551 || RMSE_x: 25.166 || RMSE_y: 25.510 || Cost: 21.645 || Recall: 0.921 || Precision: 0.999 || BatchNr.: 14200 || Time Upd.: 881.1 ms 
 || Count Cost: 28.1690 || Localization Cost: 6.8022 || Bg Cost: 0.0339 || P_locs Cost: 26.0027saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.244 || 35.835 || 63.551 || 14200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20003
JoR: 2.611 || Eff_3d: 66.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20011
JoR: 2.605 || Eff_3d: 65.723 || Jaccard: 0.930 || Factor: 718.015 || RMSE_lat: 35.687 || RMSE_ax: 62.810 || RMSE_x: 25.476 || RMSE_y: 24.990 || Cost: 18.599 || Recall: 0.931 || Precision: 0.999 || BatchNr.: 15400 || Time Upd.: 875.4 ms 
 || Count Cost: 26.7408 || Localization Cost: 4.7691 || Bg Cost: 0.0265 || P_locs Cost: 25.4457saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.723 || 35.687 || 62.810 || 15400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19732
JoR: 2.585 || Eff_3d: 65.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19740
JoR: 2.589 || Eff_3d: 65.838 || Jaccard: 0.917 || Factor: 672.531 || RMSE_lat: 35.424 || RMSE_ax: 61.712 || RMSE_x: 25.051 || RMSE_y: 25.046 || Cost: 17.887 || Recall: 0.918 || Precision: 0.999 || BatchNr.: 16600 || Time Upd.: 874.1 ms 
 || Count Cost: 26.6005 || Localization Cost: 4.1338 || Bg Cost: 0.0266 || P_locs Cost: 25.2187saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.838 || 35.424 || 61.712 || 16600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19970
JoR: 2.614 || Eff_3d: 66.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19971
JoR: 2.581 || Eff_3d: 65.784 || Jaccard: 0.928 || Factor: 705.744 || RMSE_lat: 35.970 || RMSE_ax: 61.868 || RMSE_x: 25.689 || RMSE_y: 25.178 || Cost: 18.436 || Recall: 0.929 || Precision: 0.999 || BatchNr.: 17800 || Time Upd.: 892.9 ms 
 || Count Cost: 26.8187 || Localization Cost: 4.5455 || Bg Cost: 0.0271 || P_locs Cost: 25.4344saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 65.784 || 35.970 || 61.868 || 17800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19956
JoR: 2.558 || Eff_3d: 64.

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 19901
JoR: 2.546 || Eff_3d: 64.529 || Jaccard: 0.924 || Factor: 704.127 || RMSE_lat: 36.283 || RMSE_ax: 65.995 || RMSE_x: 25.438 || RMSE_y: 25.872 || Cost: 17.968 || Recall: 0.925 || Precision: 0.998 || BatchNr.: 19000 || Time Upd.: 868.0 ms 
 || Count Cost: 26.3049 || Localization Cost: 4.3107 || Bg Cost: 0.0267 || P_locs Cost: 25.0290saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 64.529 || 36.283 || 65.995 || 19000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21494FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20003
JoR: 2.619 || Eff_3d: 65.

In [ ]:
1 + 1

In [13]:
from cellbgnet.utils.plot_funcs import plot_train_record

In [14]:
plot_train_record(model)